# DQN Implmentation

In [2]:
# https://huggingface.co/blog/deep-rl-dqn

### Running it without Training

In [12]:
import gymnasium as gym
import highway_env
from matplotlib import pyplot as plt
# %matplotlib inline

# env = gym.make('highway-v0', render_mode='rgb_array')
# env.reset()
# for _ in range(5):
# # while True:
#     action = env.unwrapped.action_type.actions_indexes["IDLE"]
#     obs, reward, done, truncated, info = env.step(action)
#     env.render()

# plt.imshow(env.render())
# plt.show()

2024-12-17 16:24:05.748 python[2021:12877892] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-17 16:24:05.748 python[2021:12877892] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


## Running Their DQN

In [1]:
import gymnasium as gym
import highway_env
from stable_baselines3 import DQN


# Visualization utils
%load_ext tensorboard
import sys
from tqdm.notebook import trange
# !pip install tensorboardx gym pyvirtualdisplay
# doesn't work cause not linux
# !apt-get install -y xvfb ffmpeg
# !git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
# sys.path.insert(0, '/content/HighwayEnv/scripts/')
# from utils import record_videos, show_videos

In [2]:
import torch


### Learning using existing model

In [11]:
# for MLP
config = {
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 15,
        "features": ["presence", "x", "y", "vx", "vy", "cos_h", "sin_h"],
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-20, 20],
            "vy": [-20, 20]
        },
        "absolute": False,
        "order": "sorted"
    }
}

# For CNN
config = {
    "observation": {
        "type": "GrayscaleObservation",
        "observation_shape": (128, 64),
        "stack_size": 4,
        "weights": [0.2989, 0.5870, 0.1140],  # weights for RGB conversion
        "scaling": 1.75,
    },
    "policy_frequency": 2
}

env = gym.make("highway-fast-v0", render_mode='rgb_array', config=config)
print(env.unwrapped.config)
mps_device = torch.device("mps")

model = DQN('CnnPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=15000,
              learning_starts=200,
              batch_size=32,
              gamma=0.8,
              device=mps_device,
              train_freq=1,
              gradient_steps=1,
              target_update_interval=50,
              verbose=1,
              tensorboard_log="highway_dqn/")

model.learn(int(2e2))

{'observation': {'type': 'GrayscaleObservation', 'observation_shape': (128, 64), 'stack_size': 4, 'weights': [0.2989, 0.587, 0.114], 'scaling': 1.75}, 'action': {'type': 'DiscreteMetaAction'}, 'simulation_frequency': 5, 'policy_frequency': 2, 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle', 'screen_width': 600, 'screen_height': 150, 'centering_position': [0.3, 0.5], 'scaling': 5.5, 'show_trajectories': False, 'render_agent': True, 'offscreen_rendering': False, 'manual_control': False, 'real_time_rendering': False, 'lanes_count': 3, 'vehicles_count': 20, 'controlled_vehicles': 1, 'initial_lane_id': None, 'duration': 30, 'ego_spacing': 1.5, 'vehicles_density': 1, 'collision_reward': -1, 'right_lane_reward': 0.1, 'high_speed_reward': 0.4, 'lane_change_reward': 0, 'reward_speed_range': [20, 30], 'normalize_reward': True, 'offroad_terminal': False}
Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_9
----

In [26]:
model.save("highway_dqn/model")

NameError: name 'model' is not defined

In [35]:
# Load and test saved model
model = DQN.load("highway_dqn/model")

# while True:
for i in range(10):
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()


## Own Implementations

### DQN

Chose to first make naive work and then make the normal work


The following is the pesudocode that will be followed when creating the DQN

Useful: https://www.youtube.com/watch?v=RVMpm86equc&list=PL58zEckBH8fCMIVzQCRSZVPUp3ZAVagWi&index=2

https://github.com/saashanair/rl-series/tree/master/dqn

https://github.com/johnnycode8/gym_solutions/blob/main/frozen_lake_dql.py

<img src="DQN.png" style="width: 900px;" align="left"/>




In [ ]:
import gymnasium as gym
import highway_env
import numpy as np
import random
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist

# Define model
class DQNNetwork(nn.Module):
    def __init__(self, in_states, h1_nodes, out_actions):
        super(DQNNetwork, self).__init__()

        # Define network layers
        self.fc1 = nn.Linear(in_states, h1_nodes)   # first fully connected layer
        self.out = nn.Linear(h1_nodes, out_actions) # output layer w
        
        self.optimizer = optim.Adam(self.parameters(), lr=1e-3)


    def forward(self, x):
        x = F.relu(self.fc1(x)) # Apply rectified linear unit (ReLU) activation
        x = self.out(x)         # Calculate output
        return x
    
# Define memory for Experience Replay
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer_state = []
        self.buffer_action = []
        self.buffer_next_state = []
        self.buffer_reward = []
        self.buffer_done = []
        self.idx = 0
        self.memory = []
    
    def store(self, state, action, next_state, reward, done):
        if len(self.buffer_state) < self.capacity:
            self.buffer_state.append(state)
            self.buffer_action.append(action)
            self.buffer_next_state.append(next_state)
            self.buffer_reward.append(reward)
            self.buffer_done.append(done)
        else:
            self.buffer_state[self.idx] = state
            self.buffer_action[self.idx] = action
            self.buffer_next_state[self.idx] = next_state
            self.buffer_reward[self.idx] = reward
            self.buffer_done[self.idx] = done

        self.idx = (self.idx+1)%self.capacity # for circular memory


    def sample(self, batch_size):
        if batch_size >  len(self.buffer_state):
            batch_size = len(self.buffer_state)
        
        indices_to_sample = random.sample(range(len(self.buffer_state)), batch_size)
        states = torch.from_numpy(np.array(self.buffer_state)[indices_to_sample]).float()
        actions = torch.from_numpy(np.array(self.buffer_action)[indices_to_sample])
        next_states = torch.from_numpy(np.array(self.buffer_next_state)[indices_to_sample]).float()
        rewards = torch.from_numpy(np.array(self.buffer_reward)[indices_to_sample]).float()
        done = torch.from_numpy(np.array(self.buffer_done)[indices_to_sample])

        return states, actions, next_states, rewards, done

    def __len__(self):
        return len(self.memory)

class DQNAgent:
    def __init__(self, memory_capacity=5000):
        # weights
        self.w1 = []
        
        # TODO: create the convolution network that takes the state and returns the possible state-actions
        self.q_net = {}
        self.w2 = []
        self.q_target_net = self.q_net # the same
        self.policy = {}

        self.epsilon = 0.2
        self.discount = 0.01
        self.batch_size = 1
        
        # the weights of q1 and q2 should be the same
        self.memory_capacity = memory_capacity
        self.memory = {} # this is the memory buffer -> setting a limit
        self.device = torch.device("mps")
    
    def initialize_neural_networks(self, env):
        # the lanes
        self.num_states = env.observation_space.shape[0]
        # print(env.action_space.sample())
        self.num_actions = env.action_space.n

        self.q_net = DQNNetwork(self.num_states, self.num_states, self.num_actions)
        self.q_target_net = DQNNetwork(self.num_states, self.num_states, self.num_actions)
        self.update_target_network()
    
    def update_target_network(self):
        # make the weights and biases the same
        self.q_target_net.load_state_dict(self.q_net.state_dict())
    
    def learn(self, env):
        self.initialize_neural_networks(env)
        
        self.memory = ReplayMemory(capacity=self.memory_capacity)

        env.reset()
        num_episodes = 100
        prefill_num = 2

        self.prefill_memory(env, prefill_num)

        for _ in range(num_episodes):
            state = env.reset()[0]
            
            # True when agent reaches the end states (colliding or passing the time)
            done = False 
            # True when agent takes more than 100 actions    
            truncated = False  
            while(not done and not truncated):
            # for i in range(2):
                # choose best action
                action = self.get_action(state)
                next_state, reward, done, truncated, info = env.step(action)
                self.memory.store(state=state, action=action, next_state=next_state, reward=reward, done=done)
                self.experience_replay()
                
                self.update_target_network()
                
                state = next_state
                # env.render()


    # either the policies are able to get miltuple actions and into the NN or the input of NN should be able to handle all of these
    # output (one of): {0: 'LANE_LEFT', 1: 'IDLE', 2: 'LANE_RIGHT', 3: 'FASTER', 4: 'SLOWER'}
    def get_action(self, state):
        # print(state)
        print(random.random())
        if random.random() <= self.epsilon: # amount of exploration reduces with the epsilon value
            return random.randrange(self.num_actions)

        if not torch.is_tensor(state):
            state = torch.tensor(np.array([state]), dtype=torch.float32)[0]

        with torch.no_grad():
        # pick the action with maximum Q-value in the Q-network 
            action = self.q_net(state)
        # these actions are discrete, return index that has highest Q
        # TODO: understand which we should be returning
        return torch.argmax(action[0]).item() 

    def experience_replay(self):
        states, actions, next_states, rewards, dones = self.memory.sample(self.batch_size)
        
        q_pred = self.q_net(states[0])
        # q value of the action taken
        q_pred = q_pred.gather(1, actions.view(-1, 1)) 
        
        q_target = self.q_target_net(next_states)[0]
        q_target = q_target.gather(1, actions.view(-1, 1))
        
        # setting Q(s',a') to 0 when the current state is a terminal state
        q_target[dones] = 0.0 
        
        y_j = rewards + (self.discount * q_target)
        y_j = y_j.view(-1, 1)
        
        # calculate the loss as the mean-squared error of yj and qpred
        self.q_net.optimizer.zero_grad()
        loss = F.mse_loss(y_j, q_pred).mean()
        loss.backward()
        self.q_net.optimizer.step()
        
    def prefill_memory(self, env, prefill_num):
        print("prefilling memory")
        for _ in range(prefill_num):
            done = False
            state = env.reset()[0]

            while not done:
                action = env.action_space.sample()
                next_state, reward, done, truncated, info = env.step(action)
                dqn_agent.memory.store(state=state, 
                                    action=action, 
                                    next_state=next_state, 
                                    reward=reward, 
                                    done=done)
        print("prefilling memory done")
        
    def gradient_descent(self):
        print("gradient descent")


dqn_agent = DQNAgent()
env = gym.make('highway-v0', render_mode='rgb_array')

dqn_agent.learn(env)

prefilling memory
prefilling memory done
0.27380601576997743
0
0.6473449764390355
4
0.734225787617938
0
0.841293428297414
3
0.8448108303210403
3
0.4744495081221871
0
0.4232309275560028
0
0.2508689617484974
0
0.6264961377968485
4
0.9910431037019017
0
0.6320532048588275
0
0.015513337256023552
0
0.5848186649866909
0
0.9780627005664049
3
0.5241173948086301
0
0.6746921725558953
0
0.7147176418267616
0
0.20677687624541108
0
0.735827875555674
0
0.6680463245148117
0
0.9066301795489136
0
0.6433628290430311
0
0.4263533297842509
0
0.730611514532331
0
0.16890957487976788
0
0.06595292130928965
0
0.16958175135517595
0
0.585601786457758
0
0.38487476104284246
4
0.9926282828473887
0
0.2957917421574152
0
0.32755972368877606
0
0.8360239483343422
4
0.09467573536323626
0
0.6123068179078854
0
0.17915599291917117
0
0.3077712730749379
0
0.9708241000056307
0
0.9958705499425806
0
0.36426352141387597
0
0.3283602559273415
2
0.04134654842893437
0
0.4608859985495646
2
0.2002758043148688
0
0.28965881584679787
0
0.232

KeyboardInterrupt: 

## Compare with other model (also their DQN)



In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
import torch
from torch import nn
import torch.nn.functional as F

# Define model
class DQN(nn.Module):
    def __init__(self, in_states, h1_nodes, out_actions):
        super().__init__()

        # Define network layers
        self.fc1 = nn.Linear(in_states, h1_nodes)   # first fully connected layer
        self.out = nn.Linear(h1_nodes, out_actions) # ouptut layer w

    def forward(self, x):
        x = F.relu(self.fc1(x)) # Apply rectified linear unit (ReLU) activation
        x = self.out(x)         # Calculate output
        return x

# Define memory for Experience Replay
class ReplayMemory():
    def __init__(self, maxlen):
        self.memory = deque([], maxlen=maxlen)
    
    def store(self, transition):
        self.memory.append(transition)

    def sample(self, sample_size):
        return random.sample(self.memory, sample_size)

    def __len__(self):
        return len(self.memory)

# FrozeLake Deep Q-Learning
class FrozenLakeDQL():
    # Hyperparameters (adjustable)
    learning_rate_a = 0.001         # learning rate (alpha)
    discount_factor_g = 0.9         # discount rate (gamma)    
    network_sync_rate = 10          # number of steps the agent takes before syncing the policy and target network
    replay_memory_size = 1000       # size of replay memory
    mini_batch_size = 32            # size of the training data set sampled from the replay memory

    # Neural Network
    loss_fn = nn.MSELoss()          # NN Loss function. MSE=Mean Squared Error can be swapped to something else.
    optimizer = None                # NN Optimizer. Initialize later.

    ACTIONS = ['L','D','R','U']     # for printing 0,1,2,3 => L(eft),D(own),R(ight),U(p)

    # Train the FrozeLake environment
    def train(self, episodes, render=False, is_slippery=False):
        # Create FrozenLake instance
        env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=is_slippery, render_mode='human' if render else None)
        num_states = env.observation_space.n
        num_actions = env.action_space.n
        
        epsilon = 1 # 1 = 100% random actions
        memory = ReplayMemory(self.replay_memory_size)

        # Create policy and target network. Number of nodes in the hidden layer can be adjusted.
        policy_dqn = DQN(in_states=num_states, h1_nodes=num_states, out_actions=num_actions)
        target_dqn = DQN(in_states=num_states, h1_nodes=num_states, out_actions=num_actions)

        # Make the target and policy networks the same (copy weights/biases from one network to the other)
        target_dqn.load_state_dict(policy_dqn.state_dict())

        print('Policy (random, before training):')
        self.print_dqn(policy_dqn)

        # Policy network optimizer. "Adam" optimizer can be swapped to something else. 
        self.optimizer = torch.optim.Adam(policy_dqn.parameters(), lr=self.learning_rate_a)

        # List to keep track of rewards collected per episode. Initialize list to 0's.
        rewards_per_episode = np.zeros(episodes)

        # List to keep track of epsilon decay
        epsilon_history = []

        # Track number of steps taken. Used for syncing policy => target network.
        step_count=0
            
        for i in range(episodes):
            state = env.reset()[0]  # Initialize to state 0
            terminated = False      # True when agent falls in hole or reached goal
            truncated = False       # True when agent takes more than 200 actions    

            # Agent navigates map until it falls into hole/reaches goal (terminated), or has taken 200 actions (truncated).
            while(not terminated and not truncated):

                # Select action based on epsilon-greedy
                if random.random() < epsilon:
                    # select random action
                    action = env.action_space.sample() # actions: 0=left,1=down,2=right,3=up
                else:
                    # select best action            
                    with torch.no_grad():
                        action = policy_dqn(self.state_to_dqn_input(state, num_states)).argmax().item()

                # Execute action
                new_state,reward,terminated,truncated,_ = env.step(action)

                # Save experience into memory
                memory.append((state, action, new_state, reward, terminated)) 

                # Move to the next state
                state = new_state

                # Increment step counter
                step_count+=1

            # Keep track of the rewards collected per episode.
            if reward == 1:
                rewards_per_episode[i] = 1

            # Check if enough experience has been collected and if at least 1 reward has been collected
            if len(memory)>self.mini_batch_size and np.sum(rewards_per_episode)>0:
                mini_batch = memory.sample(self.mini_batch_size)
                self.optimize(mini_batch, policy_dqn, target_dqn)        

                # Decay epsilon
                epsilon = max(epsilon - 1/episodes, 0)
                epsilon_history.append(epsilon)

                # Copy policy network to target network after a certain number of steps
                if step_count > self.network_sync_rate:
                    target_dqn.load_state_dict(policy_dqn.state_dict())
                    step_count=0

        # Close environment
        env.close()

        # Save policy
        torch.save(policy_dqn.state_dict(), "frozen_lake_dql.pt")

        # Create new graph 
        plt.figure(1)

        # Plot average rewards (Y-axis) vs episodes (X-axis)
        sum_rewards = np.zeros(episodes)
        for x in range(episodes):
            sum_rewards[x] = np.sum(rewards_per_episode[max(0, x-100):(x+1)])
        plt.subplot(121) # plot on a 1 row x 2 col grid, at cell 1
        plt.plot(sum_rewards)
        
        # Plot epsilon decay (Y-axis) vs episodes (X-axis)
        plt.subplot(122) # plot on a 1 row x 2 col grid, at cell 2
        plt.plot(epsilon_history)
        
        # Save plots
        plt.savefig('frozen_lake_dql.png')

    # Optimize policy network
    def optimize(self, mini_batch, policy_dqn, target_dqn):

        # Get number of input nodes
        num_states = policy_dqn.fc1.in_features

        current_q_list = []
        target_q_list = []

        for state, action, new_state, reward, terminated in mini_batch:

            if terminated: 
                # Agent either reached goal (reward=1) or fell into hole (reward=0)
                # When in a terminated state, target q value should be set to the reward.
                target = torch.FloatTensor([reward])
            else:
                # Calculate target q value 
                with torch.no_grad():
                    target = torch.FloatTensor(
                        reward + self.discount_factor_g * target_dqn(self.state_to_dqn_input(new_state, num_states)).max()
                    )

            # Get the current set of Q values
            current_q = policy_dqn(self.state_to_dqn_input(state, num_states))
            current_q_list.append(current_q)

            # Get the target set of Q values
            target_q = target_dqn(self.state_to_dqn_input(state, num_states)) 
            # Adjust the specific action to the target that was just calculated
            target_q[action] = target
            target_q_list.append(target_q)
                
        # Compute loss for the whole minibatch
        loss = self.loss_fn(torch.stack(current_q_list), torch.stack(target_q_list))

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    '''
    Converts an state (int) to a tensor representation.
    For example, the FrozenLake 4x4 map has 4x4=16 states numbered from 0 to 15. 

    Parameters: state=1, num_states=16
    Return: tensor([0., 1., 0. 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    '''
    def state_to_dqn_input(self, state:int, num_states:int)->torch.Tensor:
        input_tensor = torch.zeros(num_states)
        input_tensor[state] = 1
        return input_tensor

    # Run the FrozeLake environment with the learned policy
    def test(self, episodes, is_slippery=False):
        # Create FrozenLake instance
        env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=is_slippery, render_mode='human')
        num_states = env.observation_space.n
        num_actions = env.action_space.n

        # Load learned policy
        policy_dqn = DQN(in_states=num_states, h1_nodes=num_states, out_actions=num_actions) 
        policy_dqn.load_state_dict(torch.load("frozen_lake_dql.pt"))
        policy_dqn.eval()    # switch model to evaluation mode

        print('Policy (trained):')
        self.print_dqn(policy_dqn)

        for i in range(episodes):
            state = env.reset()[0]  # Initialize to state 0
            terminated = False      # True when agent falls in hole or reached goal
            truncated = False       # True when agent takes more than 200 actions            

            # Agent navigates map until it falls into a hole (terminated), reaches goal (terminated), or has taken 200 actions (truncated).
            while(not terminated and not truncated):  
                # Select best action   
                with torch.no_grad():
                    action = policy_dqn(self.state_to_dqn_input(state, num_states)).argmax().item()

                # Execute action
                state,reward,terminated,truncated,_ = env.step(action)

        env.close()

    # Print DQN: state, best action, q values
    def print_dqn(self, dqn):
        # Get number of input nodes
        num_states = dqn.fc1.in_features

        # Loop each state and print policy to console
        for s in range(num_states):
            #  Format q values for printing
            q_values = ''
            for q in dqn(self.state_to_dqn_input(s, num_states)).tolist():
                q_values += "{:+.2f}".format(q)+' '  # Concatenate q values, format to 2 decimals
            q_values=q_values.rstrip()              # Remove space at the end

            # Map the best action to L D R U
            best_action = self.ACTIONS[dqn(self.state_to_dqn_input(s, num_states)).argmax()]

            # Print policy in the format of: state, action, q values
            # The printed layout matches the FrozenLake map.
            print(f'{s:02},{best_action},[{q_values}]', end=' ')         
            if (s+1)%4==0:
                print() # Print a newline every 4 states

if __name__ == '__main__':

    frozen_lake = FrozenLakeDQL()
    is_slippery = False
    frozen_lake.train(1000, is_slippery=is_slippery)
    frozen_lake.test(10, is_slippery=is_slippery)

## Results
